In [2]:
'''
You are given transaction records from two systems. The first dataset contains standard fields, while the second dataset includes an extra column: payment_method.

Your task is to combine both datasets in a way that accommodates this schema difference. The missing column should be filled with null where not present. Sort the result by transaction_id (descending order) for readability.

Input Schema & Example
Dataset A
Column Name	Data Type
transaction_id	Integer
region_id	Integer
customer_id	Integer
transaction_date	String
amount_spent	Double
Dataset B (with extra column payment_method)
Column Name	Data Type
transaction_id	Integer
region_id	Integer
customer_id	Integer
transaction_date	String
amount_spent	Double
payment_method	String
Example Input Table (Dataset A)
transaction_id	region_id	customer_id	transaction_date	amount_spent
101	10	1001	2025-06-01	150.0
Example Input Table (Dataset B)
transaction_id	region_id	customer_id	transaction_date	amount_spent	payment_method
201	20	1003	2025-06-02	250.0	UPI
Output Schema
Column Name	Data Type
transaction_id	Integer
region_id	Integer
customer_id	Integer
transaction_date	String
amount_spent	Double
payment_method	String
Example Output Table
transaction_id	region_id	customer_id	transaction_date	amount_spent	payment_method
201	20	1003	2025-06-02	250.0	UPI
101	10	1001	2025-06-01	150.0	null
Starter Code
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Dataset A (no payment_method)
data_a = [
    (101, 10, 1001, "2025-06-01", 150.0),
    (102, 10, 1002, "2025-06-01", 200.0),
    (103, 10, 1005, "2025-06-02", 175.0),
    (104, 10, 1006, "2025-06-03", 220.0),
]

columns_a = ["transaction_id", "region_id", "customer_id", "transaction_date", "amount_spent"]
df_a = spark.createDataFrame(data_a, columns_a)

# Dataset B (with payment_method)
data_b = [
    (201, 20, 1003, "2025-06-02", 250.0, "UPI"),
    (202, 20, 1004, "2025-06-03", 300.0, "Card"),
    (203, 20, 1007, "2025-06-03", 180.0, "NetBanking"),
    (204, 20, 1008, "2025-06-04", 220.0, "Cash"),
]

columns_b = ["transaction_id", "region_id", "customer_id", "transaction_date", "amount_spent", "payment_method"]
df_b = spark.createDataFrame(data_b, columns_b)

# Your logic goes here to create df_result

display(df_result)
'''

'''
Note - 
Note: Why Use unionByName Over union in PySpark

In PySpark, you often need to combine two DataFrames vertically (row-wise). Spark provides two main methods:

union()

unionByName()

1️⃣ union()

Combines two DataFrames by column position.

Requires exactly the same schema and order of columns.

If one DataFrame has extra columns or a different column order, union() will fail.

Example:

# df_a columns: ["id", "name"]
# df_b columns: ["id", "name", "age"]
df_a.union(df_b)  # ❌ Fails due to extra column

2️⃣ unionByName()

Combines DataFrames by column name, not position.

Handles mismatched column order.

With allowMissingColumns=True, missing columns in either DataFrame are automatically filled with nulls.

Example:

df_result = df_a.unionByName(df_b, allowMissingColumns=True)


df_a missing payment_method → Spark fills it with null.

Columns are aligned by name, not position.

Why unionByName is preferred
Feature	                         union()	                            unionByName()
Column matching	                 By position	                        By name
Handles missing columns	         ❌ No	                                ✅ Yes (with allowMissingColumns=True)
Flexible for schema evolution	 ❌ No	                                ✅ Yes
Less error-prone	             ❌ High risk of silent misalignment     ✅ Safe alignment

How Spark aligns mismatched schemas

Identify all columns in both DataFrames.

For missing columns in one DataFrame:

Add a column with null values of the correct type.

Align columns in same order.

Concatenate rows vertically.

Visual Example:

df_a	                        df_b	                                  Result after unionByName
transaction_id, amount	        transaction_id, amount, payment_method	  transaction_id, amount, payment_method
101,            150	            201,            250,    "UPI"	          101,            150,    null
...	                            ...	                                      201,            250,    "UPI"
'''
# Initialize Spark session
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.appName('Spark Playground').getOrCreate()

# Dataset A (no payment_method)
data_a = [
    (101, 10, 1001, "2025-06-01", 150.0),
    (102, 10, 1002, "2025-06-01", 200.0),
    (103, 10, 1005, "2025-06-02", 175.0),
    (104, 10, 1006, "2025-06-03", 220.0),
]

columns_a = ["transaction_id", "region_id", "customer_id", "transaction_date", "amount_spent"]
df_a = spark.createDataFrame(data_a, columns_a)

# Dataset B (with payment_method)
data_b = [
    (201, 20, 1003, "2025-06-02", 250.0, "UPI"),
    (202, 20, 1004, "2025-06-03", 300.0, "Card"),
    (203, 20, 1007, "2025-06-03", 180.0, "NetBanking"),
    (204, 20, 1008, "2025-06-04", 220.0, "Cash"),
]

columns_b = ["transaction_id", "region_id", "customer_id", "transaction_date", "amount_spent", "payment_method"]
df_b = spark.createDataFrame(data_b, columns_b)

df_result = (
  df_a.unionByName(df_b, allowMissingColumns=True)
  .orderBy(F.col("transaction_id").desc())
)

# Display the final DataFrame
df_result.show()

+--------------+---------+-----------+----------------+------------+--------------+
|transaction_id|region_id|customer_id|transaction_date|amount_spent|payment_method|
+--------------+---------+-----------+----------------+------------+--------------+
|           204|       20|       1008|      2025-06-04|       220.0|          Cash|
|           203|       20|       1007|      2025-06-03|       180.0|    NetBanking|
|           202|       20|       1004|      2025-06-03|       300.0|          Card|
|           201|       20|       1003|      2025-06-02|       250.0|           UPI|
|           104|       10|       1006|      2025-06-03|       220.0|          NULL|
|           103|       10|       1005|      2025-06-02|       175.0|          NULL|
|           102|       10|       1002|      2025-06-01|       200.0|          NULL|
|           101|       10|       1001|      2025-06-01|       150.0|          NULL|
+--------------+---------+-----------+----------------+------------+--------